# Employee Data Cleaning Notebook
**Goal**: Transform messy, dirty data into a clean, analysis-ready dataset

File: `employee_data_with_qual.xlsx`

Common issues visible in the data:
- Inconsistent name casing & duplicates
- Invalid ages (text, extreme values)
- Negative / missing / invalid salaries
- Multiple date formats + invalid dates
- Inconsistent department names
- Missing values in different formats (NaN, empty, "not available")
- Trailing/leading spaces, typos

**Import the Required Files**

In [11]:
# Import the required files────
import pandas as pd
import numpy as np
from datetime import datetime
import re

**Load the file & Inspect the file**

In [23]:
#  Load the file & Expolatory Analysis 
df = pd.read_excel("employee_dataset_100_records.xlsx")
print("Shape:", df.shape)
display(df.head(3))
display(df.tail(3))
display(df.size)

Shape: (100, 5)


,Name,Age,Salary,Join Date,Department
0,James O'Neil,38,83577,2018-11-25,Sales
1,Sara K,36,110471,2012-06-28,Sales
2,Chris P,35,62474,2022-07-20,Sales


,Name,Age,Salary,Join Date,Department
97,Noah,NaN,56k,23-01-2020,IT
98,Ana Marie,NaN,-37192,15-02-2021,IT
99,Olivia,56,-19995,2012/01/08,marketing


500

**Statistics of Data**

In [24]:
#  Check the statisctis of data
df.describe()
df.info()
df.isnull()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        100 non-null    object
 1   Age         53 non-null     object
 2   Salary      100 non-null    object
 3   Join Date   100 non-null    object
 4   Department  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


,Name,Age,Salary,Join Date,Department
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
95,False,True,False,False,False
96,False,True,False,False,False
97,False,True,False,False,False
98,False,True,False,False,False


**Summary of Data**

In [25]:

def quick_quality_report(df):
    report = pd.DataFrame({
        'dtype': df.dtypes,
        'missing': df.isna().sum(),
        '%_missing': df.isna().mean().round(4)*100,
        'unique': df.nunique()    
    })
    
    # Most frequent value
    report['most_common'] = df.mode().iloc[0]
    
    # Show top3 values 
    for col in df.select_dtypes('object').columns:
        vc = df[col].value_counts().head(3)
        report.loc[col, 'top_3_values'] = str(vc.to_dict())
        
    return report

display(quick_quality_report(df))

,dtype,missing,%_missing,unique,most_common,top_3_values
Name,object,0,0.0,14,Daniel,"{'Daniel': 14, 'John Doe': 14, 'Chris P': 11}"
Age,object,47,47.0,33,thirty,"{'thirty': 14, 53: 3, 28: 2}"
Salary,object,0,0.0,81,not available,"{'not available': 17, '68k': 3, '51k': 2}"
Join Date,object,0,0.0,100,01-10-2015,"{'2018-11-25': 1, '2012-06-28': 1, '2022-07-20..."
Department,object,0,0.0,6,Marketing,"{'Marketing': 21, 'IT': 19, 'HR': 18}"


**Apply Cleaning** 

In [26]:
# Drop any missing value and save it in new df
df_clean = df.dropna().copy()

# Standardize column names (lowercase, strip, underscore)
df_clean.columns = df_clean.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)
df_clean.columns

Index(['name', 'age', 'salary', 'join_date', 'department'], dtype='object')

**Clean Name & Department Column**

In [27]:
df_clean['name'] = df_clean['name'].astype(str).str.strip().str.title().str.replace(r'\s+', ' ', regex=True)

# Department
df_clean['department'] = (
    df_clean['department']
    .astype(str)
    .str.strip()
    .str.title()
    .replace({
        'Hr': 'HR', 'It': 'IT', 'Sale': 'Sales', 'Marketting': 'Marketing',
        'Fin': 'Finance', 'H.R.': 'HR', '': np.nan, 'nan': np.nan
    })
)
df_clean

,name,age,salary,join_date,department
0,James O'Neil,38,83577,2018-11-25,Sales
1,Sara K,36,110471,2012-06-28,Sales
2,Chris P,35,62474,2022-07-20,Sales
3,Sophia,49,72657,2013-10-06,HR
4,David,37,84022,2024-11-13,Marketing
5,Michael,50,52958,2018-11-04,Marketing
6,John Doe,53,87154,2024-04-19,Finance
7,John Doe,45,51669,2014-03-09,HR
8,Karen,62,38963,2011-03-31,HR
9,Tommy,24,106611,2015-05-15,IT


**Clean Age Column**

In [29]:
# Clean Age Column 
age_map = {
    'thirty': 30,
    'NaN': np.nan,
    '': np.nan,
    'unknown': np.nan,
    'N/A': np.nan
}

df_clean['age'] = df_clean['age'].replace(age_map)
df_clean['age'] = pd.to_numeric(df_clean['age'], errors='coerce')
df_clean['age'] = df_clean['age'].where(
    (df_clean['age'] >= 16) & (df_clean['age'] <= 70),
    np.nan
)
df_clean

,name,age,salary,join_date,department
0,James O'Neil,38.0,83577,2018-11-25,Sales
1,Sara K,36.0,110471,2012-06-28,Sales
2,Chris P,35.0,62474,2022-07-20,Sales
3,Sophia,49.0,72657,2013-10-06,HR
4,David,37.0,84022,2024-11-13,Marketing
5,Michael,50.0,52958,2018-11-04,Marketing
6,John Doe,53.0,87154,2024-04-19,Finance
7,John Doe,45.0,51669,2014-03-09,HR
8,Karen,62.0,38963,2011-03-31,HR
9,Tommy,24.0,106611,2015-05-15,IT


**Clean Salary Column**

In [38]:
# Clean Salary Column 
# Remove currency symbols, 'k', 'not available', etc.
df_clean['salary'] = df_clean['salary'].replace({
    'not available': np.nan,
    'NaN': np.nan,
    '': np.nan,
})

df_clean['salary'] = df_clean['salary'].astype(str).str.replace('k', '000', regex=False)
df_clean['salary'] = df_clean['salary'].replace(r'[\$,]', '', regex=True)
df_clean['salary'] = pd.to_numeric(df_clean['salary'], errors='coerce')
# Convert negative salaries to positive
df_clean['salary'] = df_clean['salary'].abs()

df_clean

,name,age,salary,join_date,department,Name-Dep
0,James O'Neil,38.0,83577.0,2018-11-25,Sales,James O'Neil - Sales
1,Sara K,36.0,110471.0,2012-06-28,Sales,Sara K - Sales
2,Chris P,35.0,62474.0,2022-07-20,Sales,Chris P - Sales
3,Sophia,49.0,72657.0,2013-10-06,HR,Sophia - HR
4,David,37.0,84022.0,2024-11-13,Marketing,David - Marketing
5,Michael,50.0,52958.0,2018-11-04,Marketing,Michael - Marketing
6,John Doe,53.0,87154.0,2024-04-19,Finance,John Doe - Finance
7,John Doe,45.0,51669.0,2014-03-09,HR,John Doe - HR
8,Karen,62.0,38963.0,2011-03-31,HR,Karen - HR
9,Tommy,24.0,106611.0,2015-05-15,IT,Tommy - IT


**Clean the Date Column**

In [39]:
import calendar
from datetime import datetime

def validate_date(date_str):
    # Normalize separators
    date_str = date_str.replace('/', '-')
    parts = date_str.split('-')

    # Detect format
    if len(parts[0]) == 4:
        # YYYY-MM-DD
        year, month, day = parts
    else:
        # DD-MM-YYYY
        day, month, year = parts

    # Try to validate
    try:
        dt = datetime(int(year), int(month), int(day))
        return dt.strftime('%Y-%m-%d')
    except:
        return np.nan

# Apply to DataFrame
df_clean['join_date'] = df_clean['join_date'].apply(validate_date)
df_clean

,name,age,salary,join_date,department,Name-Dep
0,James O'Neil,38.0,83577.0,2018-11-25,Sales,James O'Neil - Sales
1,Sara K,36.0,110471.0,2012-06-28,Sales,Sara K - Sales
2,Chris P,35.0,62474.0,2022-07-20,Sales,Chris P - Sales
3,Sophia,49.0,72657.0,2013-10-06,HR,Sophia - HR
4,David,37.0,84022.0,2024-11-13,Marketing,David - Marketing
5,Michael,50.0,52958.0,2018-11-04,Marketing,Michael - Marketing
6,John Doe,53.0,87154.0,2024-04-19,Finance,John Doe - Finance
7,John Doe,45.0,51669.0,2014-03-09,HR,John Doe - HR
8,Karen,62.0,38963.0,2011-03-31,HR,Karen - HR
9,Tommy,24.0,106611.0,2015-05-15,IT,Tommy - IT


**Remove duplicates, drop missing values**

In [32]:
df_clean = df_clean.replace("", pd.NA)
df_clean = df_clean.drop_duplicates().copy()
df_clean = df_clean.dropna().copy()

df_clean

,name,age,salary,join_date,department
0,James O'Neil,38.0,83577.0,2018-11-25,Sales
1,Sara K,36.0,110471.0,2012-06-28,Sales
2,Chris P,35.0,62474.0,2022-07-20,Sales
3,Sophia,49.0,72657.0,2013-10-06,HR
4,David,37.0,84022.0,2024-11-13,Marketing
5,Michael,50.0,52958.0,2018-11-04,Marketing
6,John Doe,53.0,87154.0,2024-04-19,Finance
7,John Doe,45.0,51669.0,2014-03-09,HR
8,Karen,62.0,38963.0,2011-03-31,HR
9,Tommy,24.0,106611.0,2015-05-15,IT


**Feature Engineering**

In [42]:
# Create a composite key by concatenating cleaned identifying columns
df_clean['Name-Dep'] = (
    df_clean['name'] + ' - '  + df_clean['department'])

,name,age,salary,join_date,department,Name-Dep
1,Sara K,36.0,110471.0,2012-06-28,Sales,Sara K - Sales
9,Tommy,24.0,106611.0,2015-05-15,IT,Tommy - IT
12,James O'Neil,53.0,105290.0,2010-04-09,HR,James O'Neil - HR
6,John Doe,53.0,87154.0,2024-04-19,Finance,John Doe - Finance
14,Tim,31.0,87040.0,2011-05-14,Finance,Tim - Finance
4,David,37.0,84022.0,2024-11-13,Marketing,David - Marketing
0,James O'Neil,38.0,83577.0,2018-11-25,Sales,James O'Neil - Sales
56,Chris P,32.0,80000.0,2019-07-12,Finance,Chris P - Finance
29,John Doe,66.0,79360.0,2014-12-03,Marketing,John Doe - Marketing
17,Michael,20.0,78028.0,2013-03-17,Marketing,Michael - Marketing


**Final Sorting Accroding to Salary and Print the Data Frame**

In [44]:
df_clean = df_clean.sort_values(by='salary', ascending=False).reset_index(drop=True)

# =cleanThen drop duplicates based on this key
df_clean

,name,age,salary,join_date,department,Name-Dep
0,Sara K,36.0,110471.0,2012-06-28,Sales,Sara K - Sales
1,Tommy,24.0,106611.0,2015-05-15,IT,Tommy - IT
2,James O'Neil,53.0,105290.0,2010-04-09,HR,James O'Neil - HR
3,John Doe,53.0,87154.0,2024-04-19,Finance,John Doe - Finance
4,Tim,31.0,87040.0,2011-05-14,Finance,Tim - Finance
5,David,37.0,84022.0,2024-11-13,Marketing,David - Marketing
6,James O'Neil,38.0,83577.0,2018-11-25,Sales,James O'Neil - Sales
7,Chris P,32.0,80000.0,2019-07-12,Finance,Chris P - Finance
8,John Doe,66.0,79360.0,2014-12-03,Marketing,John Doe - Marketing
9,Michael,20.0,78028.0,2013-03-17,Marketing,Michael - Marketing
